# Função Gama


A Função Gama utilizada nesse programa tem como intuito receber os dados de posições e de doses de arquivos .csv, para tratá-los corretamente e aplicá-los em uma função matemática. 

Assim, seu retorno nos devolve os pontos como "aprovados" ou "reprovados", no sentido de verificar em quais pontos as doses são qualificadas, mediante comparação entre as calculadas e medidas.

In [5]:
# Valores de input da diferença percentual de doses e DTA
delta_Dm = float(input("Insira aqui a diferença percentual de dose: "))
delta_dM = float(input("Insira agora o valor de DTA: "))

In [6]:
import pandas as pd
import numpy as np

def gamma_index_function(rc, rm, Dc, Dm, max_Dm):
    '''
    Função Gama que será utilizada ao longo do código,
    recebendo como parâmetros as medidas de posições (cm) 
    e doses (cGy) calculadas e medidas. 
    '''
    r_diff_x = np.sqrt(rc[0]**2 + rc[1]**2)
    r_diff_y = np.sqrt(rm[0]**2 + rm[1]**2)
    r_diff = abs(r_diff_x - r_diff_y) # cálculo do módulo das posições
    D_diff = abs(Dc - Dm)/(max_Dm) # diferença entre doses
    
    F_g = np.sqrt((r_diff/delta_dM)**2 + (D_diff/delta_Dm)**2) # cálculo da função gamma

    return F_g

def position_module(row, max_Dm):
    '''
    Função para calcular o módulo de posição, que chama a
    função Gamma já passando os parâmetros necessários.
    '''
    rm_x = row['Ponto Medido X (rm_x)']
    rm_y = row['Ponto Medido Y (rm_y)']
    rc_x = row['Ponto Calculado X (rc_x)']
    rc_y = row['Ponto Calculado Y (rc_y)']
    
    rm = (rm_x, rm_y) # coordenadas da posição Rm
    rc = (rc_x, rc_y) # coordenadas da posição Rc
    
    return gamma_index_function(rc, rm, float(row['Dose Calculada (Dc)']), float(row['Dose Medida (Dm)']), max_Dm)

## *Medidas de Agosto/2022*

* Abaixo se encontra a análise dos 25 medidos realizadas em agosto/2022, onde foram calculados os valores de gamma-index dos 49 pontos calculados sobre e na vizinhança de cada ponto medido,
* Também foi feita a análise dos resultados a partir dos valores de menor gamma-index destes valores.

### Resultados - 1ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [203]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_1 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 1 - Agosto.csv'
df_augustData_sim_1 = pd.read_csv(august_data_sim_1)

# Informações do Dataframe dos dados
print(df_augustData_sim_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [8]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_1['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_aug_1 = {}

In [9]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_1), 49):
    group_df = df_augustData_sim_1.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_aug_1['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [10]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_aug_1.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_aug_1.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_aug_1)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [11]:
# Imprimir os resultados
print("Valores referentes às medidas da 1ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 1ª Simulação de Agosto

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [12]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_1 = pd.DataFrame(list(gamma_index_aug_1.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [13]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_1.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.12
1              9.0,18.0         0.07
2              0.0,18.0         0.20
3             -9.0,18.0         0.19
4            -18.0,18.0         0.12
5              18.0,9.0         0.25
6               9.0,9.0         0.26
7               0.0,9.0         0.15
8              -9.0,9.0         0.58
9             -18.0,9.0         0.16
10             18.0,0.0         0.08
11              9.0,0.0         0.40
12              0.0,0.0         0.07
13             -9.0,0.0         0.31
14            -18.0,0.0         0.55
15            18.0,-9.0         0.22
16             9.0,-9.0         0.63
17             0.0,-9.0         0.17
18            -9.0,-9.0         0.58
19           -18.0,-9.0         0.26
20           18.0,-18.0         0.13
21            9.0,-18.0         0.16
22            0.0,-18.0         0.13
23           -9.0,-18.0         0.32
24          -18.0,-18.0         0.05


### Resultados - 2ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [14]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_2 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 2 - Agosto.csv'
df_augustData_sim_2 = pd.read_csv(august_data_sim_2)

# Informações do Dataframe dos dados
print(df_augustData_sim_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [15]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_2['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_aug_2 = {}

In [16]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_2), 49):
    group_df = df_augustData_sim_2.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_aug_2['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [112]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_aug_2.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_aug_2.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_aug_2)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [113]:
# Imprimir os resultados
print("Valores referentes às medidas da 2ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 2ª Simulação de Agosto

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 23
Quantidade de valores reprovados, ou seja, maiores que um: 2

Porcentagem de valores aprovados: 92.0%
Porcentagem de valores reprovados: 8.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [114]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_2 = pd.DataFrame(list(gamma_index_aug_2.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [115]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_2.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.08
1              9.0,18.0         0.12
2              0.0,18.0         1.05
3             -9.0,18.0         0.10
4            -18.0,18.0         0.01
5              18.0,9.0         0.00
6               9.0,9.0         0.08
7               0.0,9.0         1.03
8              -9.0,9.0         0.03
9             -18.0,9.0         0.04
10             18.0,0.0         0.17
11              9.0,0.0         0.08
12              0.0,0.0         0.81
13             -9.0,0.0         0.25
14            -18.0,0.0         0.11
15            18.0,-9.0         0.04
16             9.0,-9.0         0.09
17             0.0,-9.0         0.74
18            -9.0,-9.0         0.07
19           -18.0,-9.0         0.02
20           18.0,-18.0         0.06
21            9.0,-18.0         0.11
22            0.0,-18.0         0.70
23           -9.0,-18.0         0.13
24          -18.0,-18.0         0.13


### Resultados - 3ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [116]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_3 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 3 - Agosto.csv'
df_augustData_sim_3 = pd.read_csv(august_data_sim_3)

# Informações do Dataframe dos dados
print(df_augustData_sim_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [117]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_3['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_aug_3 = {}

In [118]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_3), 49):
    group_df = df_augustData_sim_3.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_aug_3['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [119]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_aug_3.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_aug_3.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_aug_3)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [120]:
# Imprimir os resultados
print("Valores referentes às medidas da 3ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 3ª Simulação de Agosto

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [121]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_3 = pd.DataFrame(list(gamma_index_aug_3.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [122]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_3.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.06
1              9.0,18.0         0.07
2              0.0,18.0         0.13
3             -9.0,18.0         0.07
4            -18.0,18.0         0.07
5              18.0,9.0         0.15
6               9.0,9.0         0.11
7               0.0,9.0         0.19
8              -9.0,9.0         0.07
9             -18.0,9.0         0.10
10             18.0,0.0         0.76
11              9.0,0.0         0.92
12              0.0,0.0         0.70
13             -9.0,0.0         0.68
14            -18.0,0.0         0.67
15            18.0,-9.0         0.03
16             9.0,-9.0         0.16
17             0.0,-9.0         0.13
18            -9.0,-9.0         0.21
19           -18.0,-9.0         0.11
20           18.0,-18.0         0.12
21            9.0,-18.0         0.14
22            0.0,-18.0         0.10
23           -9.0,-18.0         0.23
24          -18.0,-18.0         0.12


### Resultados - 4ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [123]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_4 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 4 - Agosto.csv'
df_augustData_sim_4 = pd.read_csv(august_data_sim_4)

# Informações do Dataframe dos dados
print(df_augustData_sim_4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [124]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_4['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_aug_4 = {}

In [125]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_4), 49):
    group_df = df_augustData_sim_4.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_aug_4['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [126]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_aug_4.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_aug_4.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_aug_4)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [127]:
# Imprimir os resultados
print("Valores referentes às medidas da 4ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 4ª Simulação de Agosto

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [128]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_4 = pd.DataFrame(list(gamma_index_aug_4.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [129]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_4.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.01
1              9.0,18.0         0.07
2              0.0,18.0         0.24
3             -9.0,18.0         0.15
4            -18.0,18.0         0.06
5              18.0,9.0         0.03
6               9.0,9.0         0.32
7               0.0,9.0         0.16
8              -9.0,9.0         0.20
9             -18.0,9.0         0.06
10             18.0,0.0         0.07
11              9.0,0.0         0.52
12              0.0,0.0         0.91
13             -9.0,0.0         0.14
14            -18.0,0.0         0.23
15            18.0,-9.0         0.25
16             9.0,-9.0         0.12
17             0.0,-9.0         0.25
18            -9.0,-9.0         0.15
19           -18.0,-9.0         0.05
20           18.0,-18.0         0.08
21            9.0,-18.0         0.25
22            0.0,-18.0         0.08
23           -9.0,-18.0         0.13
24          -18.0,-18.0         0.08


### Resultados - 5ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [130]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_5 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 5 - Agosto.csv'
df_augustData_sim_5 = pd.read_csv(august_data_sim_5)

# Informações do Dataframe dos dados
print(df_augustData_sim_5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [131]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_5['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_aug_5 = {}

In [132]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_5), 49):
    group_df = df_augustData_sim_5.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_aug_5['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [133]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_aug_5.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_aug_5.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_aug_5)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [134]:
# Imprimir os resultados
print("Valores referentes às medidas da 5ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 5ª Simulação de Agosto

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [135]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_5 = pd.DataFrame(list(gamma_index_aug_5.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [136]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_5.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.05
1              9.0,18.0         0.26
2              0.0,18.0         0.17
3             -9.0,18.0         0.29
4            -18.0,18.0         0.13
5              18.0,9.0         0.25
6               9.0,9.0         0.38
7               0.0,9.0         0.11
8              -9.0,9.0         0.37
9             -18.0,9.0         0.18
10             18.0,0.0         0.07
11              9.0,0.0         0.08
12              0.0,0.0         0.57
13             -9.0,0.0         0.28
14            -18.0,0.0         0.05
15            18.0,-9.0         0.05
16             9.0,-9.0         0.11
17             0.0,-9.0         0.25
18            -9.0,-9.0         0.04
19           -18.0,-9.0         0.11
20           18.0,-18.0         0.12
21            9.0,-18.0         0.13
22            0.0,-18.0         0.24
23           -9.0,-18.0         0.04
24          -18.0,-18.0         0.10


### Resultados - 6ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [137]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_6 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 6 - Agosto.csv'
df_augustData_sim_6 = pd.read_csv(august_data_sim_6)

# Informações do Dataframe dos dados
print(df_augustData_sim_6.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [138]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_6['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_aug_6 = {}

In [139]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_6), 49):
    group_df = df_augustData_sim_6.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_aug_6['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [140]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_aug_6.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_aug_6.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_aug_6)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [141]:
# Imprimir os resultados
print("Valores referentes às medidas da 6ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 6ª Simulação de Agosto

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [142]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_6 = pd.DataFrame(list(gamma_index_aug_6.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [143]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_6.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.10
1              9.0,18.0         0.26
2              0.0,18.0         0.31
3             -9.0,18.0         0.28
4            -18.0,18.0         0.11
5              18.0,9.0         0.13
6               9.0,9.0         0.36
7               0.0,9.0         0.01
8              -9.0,9.0         0.49
9             -18.0,9.0         0.08
10             18.0,0.0         0.32
11              9.0,0.0         0.60
12              0.0,0.0         0.42
13             -9.0,0.0         0.27
14            -18.0,0.0         0.39
15            18.0,-9.0         0.14
16             9.0,-9.0         0.09
17             0.0,-9.0         0.15
18            -9.0,-9.0         0.17
19           -18.0,-9.0         0.08
20           18.0,-18.0         0.13
21            9.0,-18.0         0.15
22            0.0,-18.0         0.16
23           -9.0,-18.0         0.27
24          -18.0,-18.0         0.05


### Resultados - 7ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [144]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
august_data_sim_7 = './Agosto_22/Matrizes das Simulações para o Programa - Simulação 7 - Agosto.csv'
df_augustData_sim_7 = pd.read_csv(august_data_sim_7)

# Informações do Dataframe dos dados
print(df_augustData_sim_7.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [145]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_augustData_sim_7['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_aug_7 = {}

In [146]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_augustData_sim_7), 49):
    group_df = df_augustData_sim_7.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_aug_7['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [147]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_aug_7.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_aug_7.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_aug_7)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [148]:
# Imprimir os resultados
print("Valores referentes às medidas da 7ª Simulação de Agosto")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 7ª Simulação de Agosto

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [149]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_augustGammaIndex_sim_7 = pd.DataFrame(list(gamma_index_aug_7.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [150]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_augustGammaIndex_sim_7.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.06
1              9.0,18.0         0.29
2              0.0,18.0         0.47
3             -9.0,18.0         0.25
4            -18.0,18.0         0.13
5              18.0,9.0         0.24
6               9.0,9.0         0.06
7               0.0,9.0         0.27
8              -9.0,9.0         0.36
9             -18.0,9.0         0.06
10             18.0,0.0         0.53
11              9.0,0.0         0.25
12              0.0,0.0         0.38
13             -9.0,0.0         0.39
14            -18.0,0.0         0.36
15            18.0,-9.0         0.14
16             9.0,-9.0         0.39
17             0.0,-9.0         0.34
18            -9.0,-9.0         0.13
19           -18.0,-9.0         0.06
20           18.0,-18.0         0.06
21            9.0,-18.0         0.04
22            0.0,-18.0         0.50
23           -9.0,-18.0         0.04
24          -18.0,-18.0         0.12


## *Medidas de Fevereiro/2023*

* Abaixo se encontra a análise dos 25 medidos realizadas em fevereiro/2023, onde foram calculados os valores de gamma-index dos 49 pontos calculados sobre e na vizinhança de cada ponto medido,
* Também foi feita a análise dos resultados a partir dos valores de menor gamma-index destes valores.

### Resultados - 1ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [151]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_1 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 1 - Fevereiro.csv'
df_februaryData_sim_1 = pd.read_csv(february_data_sim_1)

# Informações do Dataframe dos dados
print(df_februaryData_sim_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [152]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_1['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_1 = {}

In [153]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_1), 49):
    group_df = df_februaryData_sim_1.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_1['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [154]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_1.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_1.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_1)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [155]:
# Imprimir os resultados
print("Valores referentes às medidas da 1ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 1ª Simulação de Fevereiro

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [156]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_1 = pd.DataFrame(list(gamma_index_feb_1.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [157]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_1.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.16
1              9.0,18.0         0.08
2              0.0,18.0         0.28
3             -9.0,18.0         0.20
4            -18.0,18.0         0.10
5              18.0,9.0         0.16
6               9.0,9.0         0.27
7               0.0,9.0         0.13
8              -9.0,9.0         0.51
9             -18.0,9.0         0.04
10             18.0,0.0         0.21
11              9.0,0.0         0.54
12              0.0,0.0         0.18
13             -9.0,0.0         0.15
14            -18.0,0.0         0.49
15            18.0,-9.0         0.15
16             9.0,-9.0         0.47
17             0.0,-9.0         0.13
18            -9.0,-9.0         0.53
19           -18.0,-9.0         0.04
20           18.0,-18.0         0.08
21            9.0,-18.0         0.21
22            0.0,-18.0         0.20
23           -9.0,-18.0         0.36
24          -18.0,-18.0         0.13


### Resultados - 2ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [158]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_2 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 2 - Fevereiro.csv'
df_februaryData_sim_2 = pd.read_csv(february_data_sim_2)

# Informações do Dataframe dos dados
print(df_februaryData_sim_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
 7   Unnamed: 7                0 non-null      float64
 8   Unnamed: 8                1 non-null      object 
 9   Unnamed: 9                1 non-null      object 
dtypes: float64(2), int64(6), object(2)
memory usage: 95.8+ KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [159]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_2['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_2 = {}

In [160]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_2), 49):
    group_df = df_februaryData_sim_2.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_2['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [161]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_2.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_2.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_2)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [162]:
# Imprimir os resultados
print("Valores referentes às medidas da 2ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 2ª Simulação de Fevereiro

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [163]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_2 = pd.DataFrame(list(gamma_index_feb_2.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [164]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_2.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0                 18,18         0.05
1                  9,18         0.10
2                  0,18         0.69
3                 -9,18         0.10
4                -18,18         0.06
5                  18,9         0.05
6                   9,9         0.07
7                   0,9         0.77
8                  -9,9         0.05
9                 -18,9         0.07
10                 18,0         0.19
11                  9,0         0.12
12                  0,0         0.63
13                 -9,0         0.24
14                -18,0         0.17
15                18,-9         0.08
16                 9,-9         0.07
17                 0,-9         0.73
18                -9,-9         0.03
19               -18,-9         0.04
20               18,-18         0.02
21                9,-18         0.07
22                0,-18         0.90
23               -9,-18         0.20
24              -18,-18         0.02


### Resultados - 3ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [165]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_3 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 3 - Fevereiro.csv'
df_februaryData_sim_3 = pd.read_csv(february_data_sim_3)

# Informações do Dataframe dos dados
print(df_februaryData_sim_3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [166]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_3['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_3 = {}

In [167]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_3), 49):
    group_df = df_februaryData_sim_3.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_3['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [168]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_3.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_3.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_3)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [169]:
# Imprimir os resultados
print("Valores referentes às medidas da 3ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 3ª Simulação de Fevereiro

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [170]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_3 = pd.DataFrame(list(gamma_index_feb_3.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [171]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_3.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.05
1              9.0,18.0         0.04
2              0.0,18.0         0.25
3             -9.0,18.0         0.05
4            -18.0,18.0         0.03
5              18.0,9.0         0.14
6               9.0,9.0         0.07
7               0.0,9.0         0.18
8              -9.0,9.0         0.02
9             -18.0,9.0         0.16
10             18.0,0.0         0.69
11              9.0,0.0         0.90
12              0.0,0.0         0.84
13             -9.0,0.0         0.89
14            -18.0,0.0         0.80
15            18.0,-9.0         0.24
16             9.0,-9.0         0.09
17             0.0,-9.0         0.07
18            -9.0,-9.0         0.09
19           -18.0,-9.0         0.22
20           18.0,-18.0         0.04
21            9.0,-18.0         0.13
22            0.0,-18.0         0.24
23           -9.0,-18.0         0.16
24          -18.0,-18.0         0.10


### Resultados - 4ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [172]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_4 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 4 - Fevereiro.csv'
df_februaryData_sim_4 = pd.read_csv(february_data_sim_4)

# Informações do Dataframe dos dados
print(df_februaryData_sim_4.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [173]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_4['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_4 = {}

In [174]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_4), 49):
    group_df = df_februaryData_sim_4.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_4['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [175]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_4.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_4.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_4)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [176]:
# Imprimir os resultados
print("Valores referentes às medidas da 4ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 4ª Simulação de Fevereiro

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [177]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_4 = pd.DataFrame(list(gamma_index_feb_4.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [178]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_4.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.06
1              9.0,18.0         0.06
2              0.0,18.0         0.24
3             -9.0,18.0         0.09
4            -18.0,18.0         0.07
5              18.0,9.0         0.08
6               9.0,9.0         0.16
7               0.0,9.0         0.22
8              -9.0,9.0         0.08
9             -18.0,9.0         0.05
10             18.0,0.0         0.23
11              9.0,0.0         0.64
12              0.0,0.0         0.84
13             -9.0,0.0         0.26
14            -18.0,0.0         0.25
15            18.0,-9.0         0.25
16             9.0,-9.0         0.10
17             0.0,-9.0         0.25
18            -9.0,-9.0         0.18
19           -18.0,-9.0         0.10
20           18.0,-18.0         0.12
21            9.0,-18.0         0.26
22            0.0,-18.0         0.20
23           -9.0,-18.0         0.17
24          -18.0,-18.0         0.06


### Resultados - 5ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [179]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_5 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 5 - Fevereiro.csv'
df_februaryData_sim_5 = pd.read_csv(february_data_sim_5)

# Informações do Dataframe dos dados
print(df_februaryData_sim_5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [180]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_5['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_5 = {}

In [181]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_5), 49):
    group_df = df_februaryData_sim_5.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_5['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [182]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_5.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_5.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_5)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [183]:
# Imprimir os resultados
print("Valores referentes às medidas da 5ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 5ª Simulação de Fevereiro

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [184]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_5 = pd.DataFrame(list(gamma_index_feb_5.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [185]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_5.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.05
1              9.0,18.0         0.21
2              0.0,18.0         0.41
3             -9.0,18.0         0.30
4            -18.0,18.0         0.12
5              18.0,9.0         0.28
6               9.0,9.0         0.30
7               0.0,9.0         0.49
8              -9.0,9.0         0.29
9             -18.0,9.0         0.60
10             18.0,0.0         0.23
11              9.0,0.0         0.27
12              0.0,0.0         0.50
13             -9.0,0.0         0.11
14            -18.0,0.0         0.20
15            18.0,-9.0         0.06
16             9.0,-9.0         0.08
17             0.0,-9.0         0.16
18            -9.0,-9.0         0.03
19           -18.0,-9.0         0.07
20           18.0,-18.0         0.06
21            9.0,-18.0         0.05
22            0.0,-18.0         0.31
23           -9.0,-18.0         0.05
24          -18.0,-18.0         0.12


### Resultados - 6ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [186]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_6 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 6 - Fevereiro.csv'
df_februaryData_sim_6 = pd.read_csv(february_data_sim_6)

# Informações do Dataframe dos dados
print(df_februaryData_sim_6.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [187]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_6['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_6 = {}

In [188]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_6), 49):
    group_df = df_februaryData_sim_6.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_6['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [189]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_6.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_6.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_6)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [190]:
# Imprimir os resultados
print("Valores referentes às medidas da 6ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 6ª Simulação de Fevereiro

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [191]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_6 = pd.DataFrame(list(gamma_index_feb_6.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [192]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_6.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.14
1              9.0,18.0         0.14
2              0.0,18.0         0.23
3             -9.0,18.0         0.33
4            -18.0,18.0         0.13
5              18.0,9.0         0.32
6               9.0,9.0         0.28
7               0.0,9.0         0.26
8              -9.0,9.0         0.36
9             -18.0,9.0         0.20
10             18.0,0.0         0.17
11              9.0,0.0         0.35
12              0.0,0.0         0.03
13             -9.0,0.0         0.39
14            -18.0,0.0         0.46
15            18.0,-9.0         0.26
16             9.0,-9.0         0.09
17             0.0,-9.0         0.00
18            -9.0,-9.0         0.19
19           -18.0,-9.0         0.07
20           18.0,-18.0         0.06
21            9.0,-18.0         0.33
22            0.0,-18.0         0.48
23           -9.0,-18.0         0.29
24          -18.0,-18.0         0.06


### Resultados - 7ª Simulação

#### Leitura dos dados

* Lê os dados do arquivo .csv com os dados do mês de Agosto.

In [193]:
# Leitura dos dados do mês de Agosto refente à 1ª Simulação e criação do Dataframe
february_data_sim_7 = './Fevereiro_23/Matrizes das Simulações para o Programa - Simulação 7 - Fevereiro.csv'
df_februaryData_sim_7 = pd.read_csv(february_data_sim_7)

# Informações do Dataframe dos dados
print(df_februaryData_sim_7.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TLD                       25 non-null     float64
 1   Dose Medida (Dm)          1225 non-null   int64  
 2   Dose Calculada (Dc)       1225 non-null   int64  
 3   Ponto Medido X (rm_x)     1225 non-null   int64  
 4   Ponto Medido Y (rm_y)     1225 non-null   int64  
 5   Ponto Calculado X (rc_x)  1225 non-null   int64  
 6   Ponto Calculado Y (rc_y)  1225 non-null   int64  
dtypes: float64(1), int64(6)
memory usage: 67.1 KB
None


#### Valores de Gamma-Index

* Analisa os 49 pontos calculados para cada ponto medido calculando o gamma-index para cada um desses pontos,
* Retorna o menor gamma-index dos calculados para cada ponto medido, com seu respectivo ponto medido.

In [194]:
# Obtém o valor máximo dentre as doses medidas
max_Dm = df_februaryData_sim_7['Dose Medida (Dm)'].max()

# Criação do dicionário que irá salvar os valores gamma-index para cada ponto
gamma_index_feb_7 = {}

In [195]:
# Calcula os valores de gamma-index de cada ponto e salva no dicionário os menores valores
for i in range(0, len(df_februaryData_sim_7), 49):
    group_df = df_februaryData_sim_7.iloc[i:i+49]  # Selecionar grupo de 49 pontos
    min_gamma = group_df.apply(lambda row: position_module(row, max_Dm), axis=1).min()
       
    # Salva o menor valor de gamma-index no dicionário
    x_coord = group_df.iloc[0]['Ponto Medido X (rm_x)']
    y_coord = group_df.iloc[0]['Ponto Medido Y (rm_y)']
    gamma_index_feb_7['{},{}'.format(x_coord, y_coord)] = min_gamma.round(2)

#### Análise dos valores obtidos

* Retorna quais valores estão 'acima' e 'abaixo ou igual' a 1, classificando-os como 'pontos reprovados' e 'pontos aprovados', respectivamente,
* Também cria um dataframe dos pontos analisados com seus respectivos gamma-index.

In [196]:
# Contar a quantidade de valores menores ou iguais a 1 e a quantidade de valores maiores que 1
menores_ou_iguais_que_um = sum(1 for valor in gamma_index_feb_7.values() if valor <= 1)

maiores_que_um = sum(1 for valor in gamma_index_feb_7.values() if valor > 1)

# Total de valores do gamma-index
total_valores = len(gamma_index_feb_7)

# Calcular as porcentagens dos valores acima
porcentagem_menores_que_um = (menores_ou_iguais_que_um / total_valores) * 100

porcentagem_maiores_que_um = (maiores_que_um / total_valores) * 100

In [197]:
# Imprimir os resultados
print("Valores referentes às medidas da 7ª Simulação de Fevereiro")
print()
print("Total de pontos analisados:", total_valores)
print(f"Diferença de percentual de dose: {delta_Dm*100}%")
print(f"DTA utilizado: {delta_dM} mm")
print()
print("Quantidade de valores aprovados, ou seja, menores ou iguais a um:", menores_ou_iguais_que_um)
print("Quantidade de valores reprovados, ou seja, maiores que um:", maiores_que_um)
print()
print(f"Porcentagem de valores aprovados: {porcentagem_menores_que_um}%")
print(f"Porcentagem de valores reprovados: {porcentagem_maiores_que_um}%")

Valores referentes às medidas da 7ª Simulação de Fevereiro

Total de pontos analisados: 25
Diferença de percentual de dose: 3.0%
DTA utilizado: 3.0 mm

Quantidade de valores aprovados, ou seja, menores ou iguais a um: 25
Quantidade de valores reprovados, ou seja, maiores que um: 0

Porcentagem de valores aprovados: 100.0%
Porcentagem de valores reprovados: 0.0%


#### Visualização dos dados tratados

* Visualiza tabela final para cada simulação, com os pontos medido e seu menor valor de gamma-index calculado.

In [198]:
# Cria um dataframe para os valores obtidos da primeira simulação
df_februaryGammaIndex_sim_7 = pd.DataFrame(list(gamma_index_feb_7.items()), columns=['Pontos Medidos (X,Y)', 'Gamma Index'])

In [199]:
# Visualiza o DataFrame com os devidos tratamentos
print(df_februaryGammaIndex_sim_7.head(25))

   Pontos Medidos (X,Y)  Gamma Index
0             18.0,18.0         0.12
1              9.0,18.0         0.30
2              0.0,18.0         0.28
3             -9.0,18.0         0.28
4            -18.0,18.0         0.15
5              18.0,9.0         0.04
6               9.0,9.0         0.42
7               0.0,9.0         0.51
8              -9.0,9.0         0.56
9             -18.0,9.0         0.15
10             18.0,0.0         0.46
11              9.0,0.0         0.29
12              0.0,0.0         0.36
13             -9.0,0.0         0.51
14            -18.0,0.0         0.56
15            18.0,-9.0         0.26
16             9.0,-9.0         0.14
17             0.0,-9.0         0.27
18            -9.0,-9.0         0.20
19           -18.0,-9.0         0.05
20           18.0,-18.0         0.09
21            9.0,-18.0         0.34
22            0.0,-18.0         0.18
23           -9.0,-18.0         0.07
24          -18.0,-18.0         0.13


## *Salvando os Resultados*

* Abaixo os resultados de cada mês são salvos em um arquivo no formato .xlsx (Excel), com os resultados das simulações em abas diferentes.

### Agosto

In [200]:
# Criando um arquivo em formato Excel para os resultados referentes ao mês de agosto
with pd.ExcelWriter('Resultados_Analises_Programa_Agosto.xlsx', engine='xlsxwriter') as writer:

    # Salvando os DataFrames de cada simulação em uma aba chamada diferente
    df_augustGammaIndex_sim_1.to_excel(writer, sheet_name='Resultados - 1ª Sim_Agosto', index=False)
    
    df_augustGammaIndex_sim_2.to_excel(writer, sheet_name='Resultados - 2ª Sim_Agosto', index=False)
    
    df_augustGammaIndex_sim_3.to_excel(writer, sheet_name='Resultados - 3ª Sim_Agosto', index=False)
    
    df_augustGammaIndex_sim_4.to_excel(writer, sheet_name='Resultados - 4ª Sim_Agosto', index=False)
    
    df_augustGammaIndex_sim_5.to_excel(writer, sheet_name='Resultados - 5ª Sim_Agosto', index=False)
    
    df_augustGammaIndex_sim_6.to_excel(writer, sheet_name='Resultados - 6ª Sim_Agosto', index=False)
    
    df_augustGammaIndex_sim_7.to_excel(writer, sheet_name='Resultados - 7ª Sim_Agosto', index=False)

### Fevereiro

In [201]:
# Criando um arquivo em formato Excel para os resultados referentes ao mês de fevereiro
with pd.ExcelWriter('Resultados_Analises_Programa_Fevereiro.xlsx', engine='xlsxwriter') as writer:

    # Salvando os DataFrames de cada simulação em uma aba chamada diferente
    df_februaryGammaIndex_sim_1.to_excel(writer, sheet_name='Resultados - 1ª Sim_Fevereiro', index=False)
    
    df_februaryGammaIndex_sim_2.to_excel(writer, sheet_name='Resultados - 2ª Sim_Fevereiro', index=False)
    
    df_februaryGammaIndex_sim_3.to_excel(writer, sheet_name='Resultados - 3ª Sim_Fevereiro', index=False)
    
    df_februaryGammaIndex_sim_4.to_excel(writer, sheet_name='Resultados - 4ª Sim_Fevereiro', index=False)
    
    df_februaryGammaIndex_sim_5.to_excel(writer, sheet_name='Resultados - 5ª Sim_Fevereiro', index=False)
    
    df_februaryGammaIndex_sim_6.to_excel(writer, sheet_name='Resultados - 6ª Sim_Fevereiro', index=False)
    
    df_februaryGammaIndex_sim_7.to_excel(writer, sheet_name='Resultados - 7ª Sim_Fevereiro', index=False)